<a href="https://colab.research.google.com/github/Yaweni/Vic/blob/main/Drebin_Traitement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
import glob
import zipfile
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import csv

In [21]:
!wget -q --show-progress --user drebin --password h9FN43vH --no-check-certificate https://www.sec.cs.tu-bs.de/~danarp/drebin/dataset/drebin-5.zip
!wget -q --show-progress --user drebin --password h9FN43vH --no-check-certificate https://www.sec.cs.tu-bs.de/~danarp/drebin/dataset/drebin-4.zip
!wget -q --show-progress --user drebin --password h9FN43vH --no-check-certificate https://www.sec.cs.tu-bs.de/~danarp/drebin/dataset/drebin-2%C2%BD.zip
!wget -q --show-progress --user drebin --password h9FN43vH --no-check-certificate https://www.sec.cs.tu-bs.de/~danarp/drebin/dataset/drebin-33%E2%85%93.zip
!wget -q --show-progress --user drebin --password h9FN43vH --no-check-certificate https://www.sec.cs.tu-bs.de/~danarp/drebin/dataset/drebin-1.zip
!wget -q --show-progress --user drebin --password h9FN43vH --no-check-certificate https://www.sec.cs.tu-bs.de/~danarp/drebin/dataset/drebin-0.zip
!wget https://github.com/Yaweni/NER-Lena/releases/download/Test/ops.txt
!wget -q --show-progress  https://bitbucket.org/JesusFreke/smali/downloads/baksmali-2.5.2.jar

drebin-5.zip        100%[===================>] 562.78M  9.11MB/s    in 2m 3s   
drebin-4.zip        100%[===================>]   1.17G  9.07MB/s    in 2m 15s  
drebin-2½.zip       100%[===================>]   1.21G  9.18MB/s    in 2m 24s  
drebin-33⅓.zip      100%[===================>]   1.07G  4.83MB/s    in 5m 4s   
drebin-1.zip        100%[===================>]   1.05G  9.23MB/s    in 2m 0s   
drebin-0.zip        100%[===================>]   1.10G  9.30MB/s    in 2m 13s  


In [22]:
%%capture
!unzip -P infected '*drebin*.zip'

In [23]:
!mkdir extracted

In [24]:
os.chdir('/content/')
f=glob.glob('*-*/*')
!rm -r /content/extracted*
!mkdir extracted

In [25]:



for apk in f:

  dir=os.path.basename(apk)
  try:
    zip_ref=zipfile.ZipFile(apk, 'r') 
  except:
    print("Pas ficher zip")
  else:
    os.chdir('/content/extracted/')
    try:
        zip_ref.extractall(dir)
    except:
      print('Error')
  finally:
    os.chdir('/content/')



        

Error
Error
Error
Pas ficher zip
Pas ficher zip
Pas ficher zip
Error
Error
Pas ficher zip


In [26]:
 os.chdir('/content/')


In [28]:
!rm -r smali
!mkdir smali

rm: cannot remove 'smali': No such file or directory


In [29]:
from tqdm import tqdm
from joblib import Parallel, delayed
os.sys
os.chdir('/content/extracted')
b=glob.glob('*')

for dir in tqdm(b[:50]):
  path=dir+'/*.dex'
  dest= '/content/smali/'+ dir
  !java -jar /content/baksmali-2.5.2.jar disassemble $path -o $dest


100%|██████████| 50/50 [01:19<00:00,  1.58s/it]


In [ ]:
import threading
os.chdir('/content/extracted')
b=glob.glob('*')
def func(i,t,j):
    #Perform one action here
    a=(j/t).__floor__()
    if i != (t-1):
      cut = b[i*a:(i+1)*a]
    else:
      cut = b[t*a:]
    for dir in cut:
      path=dir+'/*.dex'
      dest= '/content/smali/'+ dir
      !java -jar /content/baksmali-2.5.2.jar disassemble $path -o $dest

num=10
j=len(b)      
for i in range(10):
    threading.Thread(target=func(i,num,j)).start()



Can't find file: edc59a6223fb6af5e8a038cd32ada988e1cf4bdf154e4745ef33a0784acd131f/*.dex
Can't find file: 788b97bb7be845f373271bb92f50d6a5f2977bdad363e7a8080a23527566448f/*.dex


In [8]:
def std_codes_list(standard_codes):

    std_codes = [] #empty list which will later contain all the standard op-codes read from the ops.txt file

    with open(standard_codes,'r') as fp:
        for cnt, line in enumerate(fp): # reading each op-code in the txt file
            print(cnt,line)
            read_lines = fp.read()
            read_lines = read_lines.split("\n")
            std_codes = read_lines
    return std_codes 

In [36]:
standard_codes = "/content/ops.txt" 


app_dir= "/content/smali/"

In [15]:
def parse_smalis(app_dir,each_apk):

    smali_content = [] #Empty list that will later contain all of the content of all of the smali files.
    

    # import ipdb
    # ipdb.set_trace() 
    files = glob.glob(app_dir+each_apk+"/**/*.smali", recursive = True)
    #for root, dirs, files in os.walk(app_dir+each_apk+"/smali"): #Scanning through each file in each sub-directory
    for file in files:
            

        #if file.endswith(".smali"):

            

        #file_dest= os.path.join(root, file)
        file_dest=file
        with open(file_dest,'r') as fp:
            smali_content += fp.readlines()
    smali_content = [line.rstrip('\n').split(" ") for line in smali_content] #store the contents of a file

    return smali_content

In [16]:
def match_op_codes(std_codes, smali_content,each_apk):

    occ = ""
    for each_line in smali_content:
        for each_word in each_line:
            if (each_word) in (std_codes):
                occ+=each_word+" "
    fq= {}
    
    fq.setdefault('Apk Name',each_apk)
    fq.setdefault('Codes',occ)
    fq.setdefault('Label','Malware')
    return fq

In [37]:
std_codes = std_codes_list(standard_codes)
os.chdir('/content/smali')

apk_list = glob.glob('*')
valeurs=[]
for each_apk in apk_list:
    smali_content = parse_smalis(app_dir,each_apk)
    fq = match_op_codes(std_codes,smali_content,each_apk)
    valeurs.append(fq)
  

0 unused_F1



In [39]:

apk_info = ['Apk Name', 'Codes','Label']
os.chdir('/content/')

with open('Drebin.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = apk_info)
    writer.writeheader()
    writer.writerows(valeurs)

In [40]:

df=pd.read_csv("Drebin.csv")
table = pa.Table.from_pandas(df)
pq.write_table(table, 'Drebin.parquet', compression='BROTLI')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/Drebin.parquet /content/drive/MyDrive